# Extract and Transform the data from the CSV
three dataframes: df_landscape, df_caption, df_start_end_pos will be created from the csv-file.

In [ ]:
import pandas as pd
import os
import numpy as np
import csv

In [ ]:

# Read whole csv file and create a dataframe
csv_filename = "S_001_Daten.csv"
path_to_csv = os.path.join("SEARCH_001", "S001", csv_filename)
df_total = pd.read_csv(path_to_csv, sep=";", encoding="unicode-escape", header=None)

# seperate the dataframe in three subdataframes
## df_landscape / np_landscape
endrow_landscape_data = df_total.loc[pd.isna(df_total[0])].index[0]         # get index of a row containing NaN in column 0
df_landscape = df_total[:endrow_landscape_data]                             # get landscape part of data
df_landscape = df_landscape.astype(float)                                   # clean data
np_landscape = df_landscape.to_numpy().T                                    # to numpy

## df_caption
startrow_caption_data = endrow_landscape_data + 1
endrow_caption_data = df_total.loc[pd.isna(df_total[0])].index[1]            # get index of a row containing NaN in column 0
nrows_caption_data = endrow_caption_data - startrow_caption_data - 1
df_caption = pd.read_csv(path_to_csv, sep=";", encoding="unicode-escape", skiprows=startrow_caption_data, nrows=nrows_caption_data, usecols=["Code", "Bezeichnung", "Kosten"])
df_caption["Code"] = df_caption["Code"].astype(int)                         # clean data
df_caption["Bezeichnung"] = df_caption["Bezeichnung"].astype(str)           # clean data
df_caption["Kosten"] = df_caption["Kosten"].astype(float)                   # clean data

## df_start_end_pos
startrow_pos_data = endrow_caption_data + 1                                 # get index of a row where start and endpoint information starts
df_start_end_pos = pd.read_csv(path_to_csv, sep=";", encoding="unicode-escape", skiprows=startrow_pos_data, usecols=["Startpunkt", "Endpunkt"])
df_start_end_pos.index = ["x", "y"]                                         # transform dataframe
df_start_end_pos["Startpunkt"] = df_start_end_pos["Startpunkt"].astype(int)             # clean data
df_start_end_pos["Endpunkt"] = df_start_end_pos["Endpunkt"].astype(int)  
df_start_end_pos = df_start_end_pos - 1                                                 # clean data: x/y starts at 0 not 1
np_start_end_pos = df_start_end_pos.to_numpy().T   # to numpy

print(df_start_end_pos)
print(np_start_end_pos)

In [ ]:
df_landscape.shape # (number_row, number_col)
df_landscape

In [ ]:
df_caption

In [ ]:
df_start_end_pos

# Instantiate Grid

In [ ]:
import grid

In [ ]:
landscape = grid.Grid()

In [ ]:
landscape.np_grid[0][14]

In [ ]:
# test start end
print("start: ", landscape.start_pos)
print("end: ", landscape.end_pos)

# test name, cost and code of pos
pos = (0, 14)
print("code: ", landscape.get_code_of_pos(pos))
print("name: ", landscape.get_name_of_pos(pos))
print("cost: ", landscape.get_cost_of_pos(pos))


In [ ]:
pos = (0,0)
neighbours = landscape.get_neighbours_of_pos(pos)
for neighbour in neighbours:
    print(neighbour)
    print("code: ", landscape.get_code_of_pos(neighbour))
    print("name: ", landscape.get_name_of_pos(neighbour))
    print("cost: ", landscape.get_cost_of_pos(neighbour))
    print("----")

# Instantiate nodes and add it to the graph

In [ ]:
# Create Nodes 
import node
import graph
import grid

landscape = grid.Grid()
g = graph.Graph(grid=landscape)

In [ ]:
# show neighbour nodes of node
for n in g.nodes:
    print("node: "+ str(n.pos))
    print("neighb: " + str(n.neighbours))
    print("//")

In [ ]:
for key in g.adjac_list:
    print(key.pos)
    for node in g.adjac_list[key]:
        print(node.pos)
    print("//")

In [ ]:
# test if neighbour nodes is alright
pos = (0, 0)
n = g.get_node_by_pos(pos)
for node in g.get_neighbours_of_node(n):
    print(node.code)

In [ ]:
# gebe rechte node an
curr_node = g.get_node_by_pos((0,0))
curr_node= g.get_node_by_pos(curr_node.neighbours["right"])
curr_node = g.get_node_by_pos(curr_node.neighbours["right"])
curr_node.pos

In [ ]:
src_node = g.start_node
print(src_node.pos)
print(src_node.code)
print(src_node.name)
print(src_node.cost)

dest_node = g.end_node
print(dest_node.pos)
print(dest_node.code)
print(dest_node.name)
print(dest_node.cost)


In [ ]:
# Testing the h_score_cost_estimate function
import node
import graph
import grid

if __name__ == '__main__':
    landscape = grid.Grid()
    g = graph.Graph(grid=landscape)
    src_node = g.start_node
    dest_node = g.end_node
    dist = g.h_score_cost_estimate(src_node=src_node, dest_node=dest_node)
    print(dist)

In [ ]:
open_list = []
closed_list = []

open_list.append(g.start_node)
# get current node 
curr_node = open_list[0]
curr_index = 0

for index, item in enumerate(open_list):
    print(index, item)

#print(open_list)
open_list.pop(curr_index)
closed_list.append(curr_node)

#print(open_list)

In [ ]:
a = []
a.append(1)
a.append(2)
a.append(3)
a.reverse()
a.remove(2)
a.reverse()
a

In [ ]:
# test if neighbour nodes is alright
pos = (0, 0)
n = g.get_node_by_pos(pos)
for node in g.get_neighbours_of_node(n):
    print(node)

In [ ]:
# Testing score results of node
pos = (0, 1)
curr_node = g.get_node_by_pos(pos)

for neighbour_node in g.get_neighbours_of_node(curr_node):
    print("neigh_node at: ", neighbour_node.pos)
    # if current node is not in open_list and closed_list -> add to open list
    # also set its parent
    # and its f, g and h scores
    neighbour_node.parent = curr_node
    print("currnode gscore", curr_node.g_score)
    neighbour_node.g_score = curr_node.g_score + neighbour_node.cost
    neighbour_node.h_Score = g.h_score_cost_estimate(neighbour_node)
    neighbour_node.f_score = neighbour_node.g_score + neighbour_node.h_Score
    
    print(neighbour_node.f_score, neighbour_node.g_score, neighbour_node.h_Score)

In [ ]:
a = [(1,1), (2,2)]
for (x, y) in a:
    print(x, y)

In [1]:
import node
import graph
import grid
                                                                                                                    
landscape = grid.Grid()
g = graph.Graph(grid=landscape)

# landscape.df_grid
g.a_start_algorithm()
g.show_best_path_solution_of_a_star()

Path Found!


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,5,1,3,3,3,5,5,3,3,3,3,3,3,3,0
1,5,1,0,3,3,5,0,3,3,5,5,3,3,0,3
2,5,1,1,0,3,5,0,0,0,3,5,3,3,3,3
3,5,5,1,0,0,5,0,0,3,3,5,0,3,3,3
4,5,5,1,4,1,1,0,0,3,3,5,2,2,2,2
5,0,3,3,0,5,1,1,1,3,5,5,2,0,0,3
6,3,3,5,5,5,0,0,1,1,5,5,2,0,0,0
7,3,3,3,5,3,3,3,3,1,1,5,2,0,0,0
8,3,3,3,0,5,3,3,3,3,1,0,2,0,0,0
9,0,3,3,3,5,2,2,2,2,4,2,2,0,0,0


In [ ]:
#Importing libraries
import pandas as pd
df = pd.DataFrame({'A':[1,2,3,4,5],      # Sample dataframe for testing purpose
    'B':[11,22,33,44,55],
    'C':[111,222,333,444,555]
})

# We want to color cell at <3,2>. Coloring the background red, and text color Violet
# Custom function to color the desired cell
def styling_specific_cell(x):
    color = 'background-color: red; color: violet'
    df_styler = pd.DataFrame('', index=x.index, columns=x.columns)
    df_styler.iloc[2, 2] = color
    df_styler.iloc[1, 2] = color
    return df_styler

# Pass the row and column index as kwargs
df.style.apply(styling_specific_cell, axis = None)  # Axis set to None to work on entire dataframe

In [8]:
a = (12, 1)
start_x = a[0] - 1
start_y = a[1] - 1

start_pos = (start_x, start_y)
start_pos

(11, 0)